# Julia 日誌與例外處理

## 1. 日誌 (Logging)

日誌的記錄是透過巨集加上要記錄的訊息字串組成。

In [1]:
i = 1

@info "start"

i += 1
@debug "debug 訊息預設不會顯示輸出"

@info "end. Now i is $i." 

┌ Info: start
└ @ Main In[1]:3
┌ Info: end. Now i is 2.
└ @ Main In[1]:8


In [2]:
using Logging

### 1.1 Logger 及 Log Level

日誌的輸出方式可以透過設定 logger 來指定，logger 有下列幾種： 
- ConsoleLogger，預設值，輸出到螢幕。
- SimpleLogger，輸出到純文字檔案。
- NullLogger，不輸出。

範例中使用 SimpleLogger, 並將 log 以標準輸出 (stdout) 顯示。


日誌是否要顯示或記錄是由 Log level (等級) 來控制，主要的 Log level 等級有：
- Debug
- Info
- Warn
- Error

範例中 log level 設定為 debug。

**Logging.SimpleLogger**
- `SimpleLogger(stream=stderr, min_level=Info)`
- Simplistic logger for logging all messages with level greater than or equal to min_level to stream.

In [5]:
logger = SimpleLogger(stdout, Logging.Debug)
global_logger(logger)

@info "正常的日誌訊息"
@debug "除錯用的訊息, 預設不會顯示"

try
    @debug "除錯用的訊息, 預設不會顯示"
    println("try 區塊內容")
catch
    @error "錯誤訊息" # or error("錯誤訊息")
#     error("錯誤訊息")
end

┌ Info: 正常的日誌訊息
└ @ Main In[5]:4
┌ Debug: 除錯用的訊息, 預設不會顯示
└ @ Main In[5]:5
┌ Debug: 除錯用的訊息, 預設不會顯示
└ @ Main In[5]:8
try 區塊內容


In [10]:
error("錯誤訊息")
1 + 1

ErrorException: 錯誤訊息

In [11]:
@error "錯誤訊息但是可以繼續跑下面的東東"
1 + 1

┌ Error: 錯誤訊息但是可以繼續跑下面的東東
└ @ Main In[11]:1


2

呼叫 `min_enabled_level()` 函式查看目前的 log level 設定。

In [28]:
Logging.min_enabled_level(logger)

Debug

呼叫 `current_logger()` 函式查看目前 logger 的種類。

In [29]:
current_logger()

Base.CoreLogging.SimpleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x0000000000000450) open, 0 bytes waiting))), Debug, Dict{Any,Int64}())

### 1.2 task-local logger 與 global_logger

Logger 可以分為 global_logger 與 task-local logger。

In [30]:
# task-local logger
with_logger(logger) do
    @info("a context specific log message")
end

┌ Info: a context specific log message
└ @ Main In[30]:3


In [31]:
# global_logger
global_logger(logger)
@info "a global log message"

┌ Info: a global log message
└ @ Main In[31]:3


### 1.3 將 Log 寫入到檔案

將日誌 (Logging) 寫入文字檔案的流程如下：
1. 開啟文字檔 IO，並照上兩頁的流程建立日誌。
2. 記錄日誌訊息。
3. 呼叫 flush() 函式，會將所有都暫存的日誌訊息寫入到文字檔案中。
4. 最後記得要呼叫 close() 函式關閉檔案。

In [32]:
io = open("log.txt", "w+")

logger = SimpleLogger(io, Logging.Debug)

global_logger(logger)

@info("a global info log message")
@debug("a global debug log message")

In [33]:
flush(io)

In [34]:
close(io)

查看寫入 log 文字檔的內容。

In [35]:
f = open("log.txt")
readlines(f)

4-element Array{String,1}:
 "┌ Info: a global info log message"
 "└ @ Main In[32]:7"
 "┌ Debug: a global debug log message"
 "└ @ Main In[32]:8"

In [36]:
close(f)

## 2. 例外處理 (Exception Handling)

如果不可控制的情況發生，可能就會產生系統 Error，造成程式執行中斷，這時候我們就需要在適當的位置中，加入例外處理機制，讓我們可以捕捉到這些錯誤，並進行相對應的處理，以避免程式中斷而失去控制。

例如開啟一個不存在的檔案，會造成程式中斷，無法繼續往下進行。

In [37]:
f = open("thisfiledoesnotexist.txt")

SystemError: SystemError: opening file "thisfiledoesnotexist.txt": No such file or directory

### 2.1 try-catch-finally

使用 try-catch 例外處理。例如：
- 開啟檔案時可能產生出錯，我們將有可能需要對例外進行測試的表達式放在 try 區塊中。
- 若發生錯誤時，由 catch 區塊來處理捕捉到的錯誤，避免程式因為錯誤而異常中斷。

In [38]:
try
    f = open("thisfiledoesnotexist.txt")
catch ex
    println("檔案開啟錯誤，請檢查檔案是否存在。錯誤訊息：")
    println("Exception: ", ex)
end

檔案開啟錯誤，請檢查檔案是否存在。錯誤訊息：
Exception: SystemError("opening file \"thisfiledoesnotexist.txt\"", 2, nothing)


finally 區塊是在處理過後，不論正常或有例外產生，一定會進入的區塊。可以將處理過後的訊息，或後續要執行的動作，在 finally 區塊中定義。

In [39]:
try
    log(-10)
catch ex
    println("捕捉到例外情況，請檢查")
    println("Exception: ", ex, "\n")
finally
    println("This is finally.")
end

捕捉到例外情況，請檢查
Exception: DomainError(-10.0, "log will only return a complex result if called with a complex argument. Try log(Complex(x)).")

This is finally.


### 2.2 `throw()` 函式

如果要很明確地產生例外情況，可以使用 `throw()` 函式。

In [40]:
foo(x) = x>=0 ? sqrt(x) : throw(DomainError(x, "必須為正數"))

foo (generic function with 1 method)

In [41]:
foo(-4)

DomainError: DomainError with -4:
必須為正數

### 2.3 自定 Error

除了 Julia 提供的 Error 之外，也可以透過複合型別自訂 Error。

In [42]:
struct CustomException <: Exception 
    msg::AbstractString
end

### 2.4 `stacktrace()` 與 `backtrace()`

如果想要了解程式執行該時間點的執行堆疊，可以呼叫 `stacktrace()` 函式，尤其是在處理例外情況發生時，例如可以依此堆疊追蹤錯誤發生時的程式、函式呼叫流程與順序，並判斷錯錯可能發生的原因。

`backtrace()` 函式也是回傳堆疊追蹤的資訊，與 `stacktrace()` 不同的地方在於，`backtrace()` 會包含低階呼叫 C 語言的訊息。

In [43]:
stacktrace()

8-element Array{Base.StackTraces.StackFrame,1}:
 top-level scope at In[43]:1
 eval at boot.jl:331 [inlined]
 softscope_include_string(::Module, ::String, ::String) at SoftGlobalScope.jl:218
 execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:67
 #invokelatest#1 at essentials.jl:712 [inlined]
 invokelatest at essentials.jl:711 [inlined]
 eventloop(::ZMQ.Socket) at eventloop.jl:8
 (::IJulia.var"#15#18")() at task.jl:358

In [44]:
backtrace()

16-element Array{Union{Ptr{Nothing}, Base.InterpreterIP},1}:
 Ptr{Nothing} @0x000000000afadf9e
 Ptr{Nothing} @0x00000000668c81fc
 Ptr{Nothing} @0x00000000668c7ec1
 Ptr{Nothing} @0x00000000668c9420
 Ptr{Nothing} @0x00000000668c9dc5
 Base.InterpreterIP in top-level CodeInfo for Main at statement 0
 Ptr{Nothing} @0x00000000668e6f4d
 Ptr{Nothing} @0x00000000668e8068
 Ptr{Nothing} @0x000000002e5654a0
 Ptr{Nothing} @0x000000002e5616cd
 Ptr{Nothing} @0x00000000668bf2b8
 Ptr{Nothing} @0x00000000668bfa5a
 Ptr{Nothing} @0x000000002e5590eb
 Ptr{Nothing} @0x000000002e5592c8
 Ptr{Nothing} @0x000000002e5592e3
 Ptr{Nothing} @0x00000000668cdbcf

In [45]:
bad_function() = begin undeclared_variable end

example() = try
        bad_function()
    catch
        stacktrace()
    end

example()

9-element Array{Base.StackTraces.StackFrame,1}:
 example() at In[45]:6
 top-level scope at In[45]:8
 eval at boot.jl:331 [inlined]
 softscope_include_string(::Module, ::String, ::String) at SoftGlobalScope.jl:218
 execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:67
 #invokelatest#1 at essentials.jl:712 [inlined]
 invokelatest at essentials.jl:711 [inlined]
 eventloop(::ZMQ.Socket) at eventloop.jl:8
 (::IJulia.var"#15#18")() at task.jl:358

在上面的例子中，雖然可以正確地捕捉到 Error，但是回傳的錯誤發生行數在第 6 行，也就是 `stacktrace()` 被呼叫的那一行，對於要追蹤錯誤較不精確。下面示範搭配 `catch_backtrace()` 函式，可以看到錯誤發生的行數指向未宣告變數，也就是被呼叫函式的那一行 (第 1 行)，對於錯誤發生的來源能正確指出。

In [46]:
bad_function() = begin undeclared_variable end

example() = try
        bad_function()
    catch
        stacktrace(catch_backtrace())
    end

example()

10-element Array{Base.StackTraces.StackFrame,1}:
 bad_function at In[46]:1 [inlined]
 example() at In[46]:4
 top-level scope at In[46]:8
 eval at boot.jl:331 [inlined]
 softscope_include_string(::Module, ::String, ::String) at SoftGlobalScope.jl:218
 execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:67
 #invokelatest#1 at essentials.jl:712 [inlined]
 invokelatest at essentials.jl:711 [inlined]
 eventloop(::ZMQ.Socket) at eventloop.jl:8
 (::IJulia.var"#15#18")() at task.jl:358

下面的範例是呼叫 `Base.catch_stack()` 函式，並將在堆疊中的 Exception 及 backtrace 印出。

[註] `Base.catch_stack()` 函式是 Julia 1.1 之後推出的實驗性函式，未來名稱可能會改變。

In [47]:
bad_function() = begin undeclared_variable end

example() = try
        bad_function()
    catch
        for (ex, bt) in Base.catch_stack()
            showerror(stdout, ex, bt)
            println()
        end
    end

example()

UndefVarError: undeclared_variable not defined
Stacktrace:
 [1] bad_function at .\In[47]:1 [inlined]
 [2] example() at .\In[47]:4
 [3] top-level scope at In[47]:11
 [4] eval at .\boot.jl:331 [inlined]
 [5] softscope_include_string(::Module, ::String, ::String) at C:\Users\HSI\.julia\packages\SoftGlobalScope\cSbw5\src\SoftGlobalScope.jl:218
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\HSI\.julia\packages\IJulia\yLI42\src\execute_request.jl:67
 [7] #invokelatest#1 at .\essentials.jl:712 [inlined]
 [8] invokelatest at .\essentials.jl:711 [inlined]
 [9] eventloop(::ZMQ.Socket) at C:\Users\HSI\.julia\packages\IJulia\yLI42\src\eventloop.jl:8
 [10] (::IJulia.var"#15#18")() at .\task.jl:358
